In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import torchvision.models as models

In [2]:
from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import shutil
import time

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
train_transforms = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.RandomRotation(5),
                            transforms.RandomHorizontalFlip(0.5),
                            transforms.RandomCrop((224,224), padding = 10),
                            transforms.ToTensor()
                                      ])

In [5]:
valid_transforms = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.CenterCrop((224,224)),
                            transforms.ToTensor()
                                      ])

In [6]:
train_data = datasets.ImageFolder(root = '/content/drive/My Drive/Retail Pulse ML Assignment Data/train',transform=train_transforms)

In [7]:
fil1 = open('/content/drive/My Drive/Retail Pulse ML Assignment Data/images_family_train.txt', 'r') 
Lin1 = fil1.readlines() 

In [8]:
c = list()

In [9]:
l = 0
temp = Lin1[0][8:]
for i in range(len(Lin1)):
    if(temp == Lin1[i][8:]):
        c.append(l)
    else:
        temp = Lin1[i][8:]
        l=l+1
        c.append(l)

In [10]:
print(l)

69


In [11]:
for i in range(len(Lin1)):
    train_data.targets[i] = c[i]
print(len(train_data.targets))

3334


In [12]:
print(len(train_data.targets))

3334


In [13]:
valid_data = datasets.ImageFolder(root = '/content/drive/My Drive/Retail Pulse ML Assignment Data/val',transform=valid_transforms)
print(len(valid_data.targets))

3333


In [14]:
fil2 = open('/content/drive/My Drive/Retail Pulse ML Assignment Data/images_family_val.txt', 'r') 
Lin2 = fil2.readlines() 

In [15]:
d = list()

In [16]:
l = 0
temp = Lin2[0][8:]
for i in range(len(Lin2)):
    if(temp == Lin2[i][8:]):
        d.append(l)
    else:
        temp = Lin2[i][8:]
        l=l+1
        d.append(l)

In [17]:
for i in range(len(Lin2)):
    valid_data.targets[i] = d[i]

In [18]:
len(valid_data.targets)
l

69

In [19]:
BATCH_SIZE = 8
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
model = models.resnet50(pretrained=True).to(device)
i=0
for param in model.parameters():
  if i<154:
    param.requires_grad=False
    i=i+1
model.fc = nn.Sequential(
               nn.Linear(2048, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 100)).to(device)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,119,700 trainable parameters


In [23]:
def softmax(x):
    exp_x = torch.exp(x)
    sum_x = torch.sum(exp_x[0:70], dim=1, keepdim=True)

    return exp_x/sum_x

def log_softmax(x):
    return torch.log(softmax(x))

def CrossEntropyLoss(outputs, targets):
    num_examples = targets.shape[0]
    batch_size = outputs.shape[0]
    outputs = log_softmax(outputs[0:70])
    outputs = outputs[range(batch_size), targets]
    return - torch.sum(outputs)/num_examples
optimizer = optim.Adam(model.parameters(), lr = 1e-04)

In [24]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [25]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = CrossEntropyLoss(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [26]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = CrossEntropyLoss(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
EPOCHS = 300

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, CrossEntropyLoss, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, CrossEntropyLoss, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 98m 54s
	Train Loss: 4.606 | Train Acc: 1.74%
	 Val. Loss: 4.547 |  Val. Acc: 3.87%
Epoch: 02 | Epoch Time: 1m 15s
	Train Loss: 4.472 | Train Acc: 5.14%
	 Val. Loss: 4.359 |  Val. Acc: 4.92%
Epoch: 03 | Epoch Time: 1m 15s
	Train Loss: 4.226 | Train Acc: 6.03%
	 Val. Loss: 4.100 |  Val. Acc: 8.54%
Epoch: 04 | Epoch Time: 1m 15s
	Train Loss: 3.996 | Train Acc: 8.81%
	 Val. Loss: 3.896 |  Val. Acc: 9.14%
Epoch: 05 | Epoch Time: 1m 15s
	Train Loss: 3.820 | Train Acc: 11.54%
	 Val. Loss: 3.764 |  Val. Acc: 11.87%
Epoch: 06 | Epoch Time: 1m 16s
	Train Loss: 3.725 | Train Acc: 13.43%
	 Val. Loss: 3.699 |  Val. Acc: 12.65%
Epoch: 07 | Epoch Time: 1m 15s
	Train Loss: 3.627 | Train Acc: 14.43%
	 Val. Loss: 3.618 |  Val. Acc: 12.85%
Epoch: 08 | Epoch Time: 1m 15s
	Train Loss: 3.512 | Train Acc: 16.76%
	 Val. Loss: 3.598 |  Val. Acc: 13.58%
Epoch: 09 | Epoch Time: 1m 15s
	Train Loss: 3.435 | Train Acc: 17.16%
	 Val. Loss: 3.533 |  Val. Acc: 14.03%
Epoch: 10 | Epoch Time: 1m

KeyboardInterrupt: ignored